<a href="https://colab.research.google.com/github/quanvu0996/compet/blob/master/Test_API_VNQuant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thử nghiệm thư viện vnquant để đồng bộ dữ liệu và xây dựng mô hình dự đoán giá chứng khoán

In [16]:
!pip install git+https://github.com/phamdinhkhanh/vnquant

  Cloning https://github.com/phamdinhkhanh/vnquant to /tmp/pip-req-build-8dvqxr1s
  Running command git clone -q https://github.com/phamdinhkhanh/vnquant /tmp/pip-req-build-8dvqxr1s
     |████████████████████████████████| 6.3MB 5.3MB/s 
  Created wheel for vnquant: filename=vnquant-0.0.2-cp37-none-any.whl size=9759 sha256=e8e500318f7cfb3ec5da79be3660e23b681f8e93e11a327c88460a9ab88fd2db
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kz8jky9/wheels/70/f7/51/ab84791f03861a32e07cb59321d8911fb994eee39e08f2b77b
  Created wheel for pypandoc: filename=pypandoc-1.5-cp37-none-any.whl size=17036 sha256=1d5c435e8d9b8436b2e4392519ff661d8ea96bea3e594bbeb9b28841bca232bb
  Stored in directory: /root/.cache/pip/wheels/bb/7d/d6/2f9af55e800d37e42e546106bcbd36a86e24e725e303d17e04
Successfully built vnquant pypandoc
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [21]:
import vnquant.DataLoader as web
import pandas as pd
import numpy as np

In [29]:
loader = web.DataLoader('VND', '2020-02-02','2021-04-02', data_source='VND')
data = loader.download()
data.head()

2021-05-29 16:48:41,765 : INFO : data VND from 2020-02-02 to 2021-04-02 have already cloned!


Attributes,high,low,open,close,avg,volume
Symbols,VND,VND,VND,VND,VND,VND
date,,,,,,
2020-02-03,14.55,13.70,14.55,14.00,13.88,854260.0
2020-02-04,14.00,13.35,14.00,14.00,13.66,617580.0
2020-02-05,14.10,13.70,13.90,14.10,13.93,104530.0
2020-02-06,14.20,13.35,13.35,14.10,13.86,210290.0
2020-02-07,14.20,13.80,14.00,14.05,14.00,158540.0


In [40]:
data.columns = ['high', 'low', 'open', 'close', 'avg', 'volumne']

In [42]:
data['stdev'] = 

date
2020-02-03    14.55
2020-02-04    14.00
2020-02-05    13.90
2020-02-06    13.35
2020-02-07    14.00
              ...  
2021-03-24    28.60
2021-03-25    28.90
2021-03-26    28.35
2021-03-29    29.00
2021-03-30    29.40
Name: open, Length: 292, dtype: float64